In [1]:
from utility_functions import load_file
import pandas as pd

In [2]:
codes = load_file('fully_filtered_codes.pickle').reset_index()
codes = codes.set_index(codes['visit_link'].astype('int64'))['codes']

In [3]:
costs = load_file('summary_costs_enhanced.pickle')

In [4]:
costs.columns

Index(['died', 'initial_record_id', 'initial_year', 'age', 'homeless', 'race',
       'median_zip_income', 'payer', 'initial_discharge_quarter', 'max_year',
       'ED_revisits', 'surgery_visits', 'inpatient_visits', 'hyperemesis',
       'gender', 'marital_status', 'age_groups', 'CPT Costs', 'SID_costs',
       'Cost'],
      dtype='object')

In [5]:
costs = costs.join((codes.str.contains('7219[234]') | codes.str.contains('741[567]0') | codes.str.contains('7417[678]')).reset_index().groupby('visit_link').agg('any')['codes'].rename('Abdominal CT'), how='left')
costs['Endoscopy'] = codes.str.contains('^432[4-7]').reset_index().groupby('visit_link').agg('any')
costs['Colonoscopy'] = codes.str.contains('^453[789]').reset_index().groupby('visit_link').agg('any')
costs['Hospitalized'] = costs['inpatient_visits'] > 0

In [10]:
costs.groupby('Abdominal CT')['Cost'].agg(['mean', 'count'])

,mean,count
Abdominal CT,,
False,846.212969,589
True,1037.760777,117


In [11]:
costs.groupby('Endoscopy')['Cost'].agg(['mean', 'count'])

,mean,count
Endoscopy,,
False,877.956728,706


In [12]:
costs.groupby('Colonoscopy')['Cost'].agg(['mean', 'count'])

,mean,count
Colonoscopy,,
False,877.956728,706


In [13]:
costs.groupby('Hospitalized')['Cost'].agg(['mean', 'count'])

,mean,count
Hospitalized,,
False,367.041870,659
True,8041.635263,47
